<a href="https://colab.research.google.com/github/cedro3/yolact_edge/blob/master/YolactEdge_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# セットアップ

In [ ]:
# GPUの仕様チェック
!nvidia-smi -L

# Githubからコードをコピー
!git clone https://github.com/cedro3/yolact_edge.git
!git clone https://github.com/chentinghao/download_google_drive.git
%cd yolact_edge
!mkdir -p weights

# 重みとCOCOデータのダウンロード
! pip install --upgrade gdown
!python ../download_google_drive/download_gdrive.py 1EAzO-vRDZ2hupUJ4JFSUi40lAZ5Jo-Bp ./weights/yolact_edge_54_800000.pth
!python ../download_google_drive/download_gdrive.py 15jyd5CRJxNiA41UMjGbaSnmaytfeILfI ./calib_images_coco.zip
!unzip -q calib_images_coco.zip

# ライブラリーのインポート
import cv2
from google.colab.patches import cv2_imshow
from IPython.display import HTML
from base64 import b64encode

# 静止画を物体検出

In [ ]:
# サンプル画像を表示
im = cv2.imread("./pic/sample.jpg")
cv2_imshow(im)

In [ ]:
# 静止画から物体検出
!python eval.py --trained_model=weights/yolact_edge_54_800000.pth\
                --score_threshold=0.15 --top_k=100\
                --image=pic/sample.jpg:output_img.jpg\
                --disable_tensorrt

In [ ]:
# 検出画像を表示
im = cv2.imread("./output_img.jpg")
cv2_imshow(im)

# ビデオから物体検出

In [ ]:
# サンプルビデオの確認
mp4 = open('./video/sample.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="80%" height="80%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

In [ ]:
# ビデオから物体検出し、mp4動画を作成する
!python eval.py --trained_model=weights/yolact_edge_54_800000.pth\
                --score_threshold=0.15 --top_k=100\
                --video=video/sample.mp4:output_video.mp4\
                --disable_tensorrt

# コーデック変換
import os
import shutil 
if os.path.exists('./output.mp4'):
   os.remove('./output.mp4')
! ffmpeg -i output_video.mp4 -vcodec h264 -pix_fmt yuv420p output.mp4

In [ ]:
# mp4動画の再生
mp4 = open('./output.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="80%" height="80%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")